<a href="https://colab.research.google.com/github/samitha278/multilayer-perceptron-char-lm/blob/main/mlp_char_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files

uploaded = files.upload()


Saving names.txt to names.txt


In [ ]:
words = open("names.txt","r").read().splitlines()

In [ ]:
chars = sorted(list(set("".join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s,i in stoi.items()}
itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

### Building dataset

In [15]:
block_size = 3


xs = []
ys = []


context = [0]*block_size



for w in words:

  for ch in w + '.':

    xs.append(context)
    next = stoi[ch]
    ys.append(next)


    context = context[1:] + [next]


x = torch.tensor(xs)
y = torch.tensor(ys)

In [17]:
x.shape,y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [48]:
x[:5]

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1]])

### Embedding

In [21]:
C = torch.randn((27,2))
C

tensor([[ 0.2781,  0.4918],
        [ 1.3179,  1.4982],
        [ 0.6720, -0.9691],
        [-2.1497, -2.1738],
        [ 0.7141, -0.8387],
        [-0.1015,  1.2391],
        [-0.1621,  0.8586],
        [-1.9788,  0.5472],
        [-1.1494, -0.5262],
        [-0.3381, -0.3613],
        [-1.3245,  2.0941],
        [ 1.0029,  1.7922],
        [-0.8868, -0.0446],
        [-0.2211, -0.7242],
        [ 1.4162,  1.4105],
        [ 1.1210,  0.7991],
        [ 0.4431,  1.8579],
        [ 0.0807, -0.7559],
        [-1.5218, -0.7468],
        [ 2.1135, -1.6136],
        [-2.5826, -0.6771],
        [ 0.0170, -0.6379],
        [-0.6740,  0.7792],
        [ 0.0028, -0.2878],
        [-0.7103,  0.1821],
        [-0.6147,  0.0587],
        [ 1.1983, -0.6559]])

In [22]:
C[5]

tensor([-0.1015,  1.2391])

In [23]:
C[torch.tensor([2,3,5,5,5])]

tensor([[ 0.6720, -0.9691],
        [-2.1497, -2.1738],
        [-0.1015,  1.2391],
        [-0.1015,  1.2391],
        [-0.1015,  1.2391]])

In [25]:
C[torch.tensor([[2,3],[3,2],[5,5],[2,5]])]

tensor([[[ 0.6720, -0.9691],
         [-2.1497, -2.1738]],

        [[-2.1497, -2.1738],
         [ 0.6720, -0.9691]],

        [[-0.1015,  1.2391],
         [-0.1015,  1.2391]],

        [[ 0.6720, -0.9691],
         [-0.1015,  1.2391]]])

In [26]:
emb = C[x]
emb.shape

torch.Size([228146, 3, 2])

In [33]:
x[1],C[[0,5]],emb[1]

(tensor([0, 0, 5]),
 tensor([[ 0.2781,  0.4918],
         [-0.1015,  1.2391]]),
 tensor([[ 0.2781,  0.4918],
         [ 0.2781,  0.4918],
         [-0.1015,  1.2391]]))

### Preparing input neurons to feed hidden layer

In [44]:
emb[:5]

tensor([[[ 0.2781,  0.4918],
         [ 0.2781,  0.4918],
         [ 0.2781,  0.4918]],

        [[ 0.2781,  0.4918],
         [ 0.2781,  0.4918],
         [-0.1015,  1.2391]],

        [[ 0.2781,  0.4918],
         [-0.1015,  1.2391],
         [-0.2211, -0.7242]],

        [[-0.1015,  1.2391],
         [-0.2211, -0.7242],
         [-0.2211, -0.7242]],

        [[-0.2211, -0.7242],
         [-0.2211, -0.7242],
         [ 1.3179,  1.4982]]])

In [46]:
emb[:5,0]

tensor([[ 0.2781,  0.4918],
        [ 0.2781,  0.4918],
        [ 0.2781,  0.4918],
        [-0.1015,  1.2391],
        [-0.2211, -0.7242]])

In [47]:
torch.cat([emb[:5,0],emb[:5,1],emb[:5,2]],dim=1)

tensor([[ 0.2781,  0.4918,  0.2781,  0.4918,  0.2781,  0.4918],
        [ 0.2781,  0.4918,  0.2781,  0.4918, -0.1015,  1.2391],
        [ 0.2781,  0.4918, -0.1015,  1.2391, -0.2211, -0.7242],
        [-0.1015,  1.2391, -0.2211, -0.7242, -0.2211, -0.7242],
        [-0.2211, -0.7242, -0.2211, -0.7242,  1.3179,  1.4982]])

In [49]:
torch.cat([emb[:,0],emb[:,1],emb[:,2]],dim=1).shape

torch.Size([228146, 6])

In [51]:
torch.cat(torch.unbind(emb,1),1).shape

torch.Size([228146, 6])

torch view and storage

In [53]:
a = torch.arange(12)
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [55]:
a.storage()

/tmp/ipython-input-214256462.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  a.storage()


 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 12]

In [56]:
a.view(3,4)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [60]:
#only view has been changed memory allocation of a is same as before

# https://blog.ezyang.com/2019/05/pytorch-internals/

a.view(3,4).storage()

 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 12]

### Implementing hidden layer

In [61]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [62]:
h = torch.tanh(emb.view(-1,6) @ W1 + b1)

In [64]:
h.shape , h

(torch.Size([228146, 100]),
 tensor([[ 0.4236,  0.9887, -0.9415,  ..., -0.3292,  0.1220, -0.2913],
         [ 0.9232,  0.9943, -0.9591,  ...,  0.1207, -0.5562, -0.5327],
         [ 0.5709,  0.9917, -0.3924,  ..., -0.9487, -0.8677,  0.9805],
         ...,
         [-0.9970,  0.7361, -0.9998,  ...,  0.3016,  0.9979, -0.9869],
         [ 0.5216, -0.3370, -0.9771,  ..., -0.9691,  0.9597, -0.9063],
         [-0.9970,  0.9992, -0.9784,  ...,  0.9370,  0.9736, -0.9938]]))